<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Langgraph_Agentic_RAG_Cyber_AI_Copilot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Cyber AI Copilot for Security and Intelligence Domain**

In [1]:
!pip install --upgrade --quiet sentence-transformers langchain langchain-groq langchain-pinecone langchain-core asknews langgraph
%pip install --quiet -U "langchain-community>=0.2.16"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.

In [2]:
import os
from typing import List, Dict, Any, Optional, TypedDict
from pydantic import BaseModel
from langchain_groq import ChatGroq
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langgraph.graph import StateGraph, END
from langchain_community.tools.asknews import AskNewsSearch
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.tools import JinaSearch
from dotenv import load_dotenv
import re
from datetime import datetime, timedelta

# Load environment variables
load_dotenv()

# API Keys (hidden for security purposes)
GROQ_API_KEY = "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"
PINECONE_API_KEY = "8e15b925-3b96-497d-b20a-08d308782b83"
PINECONE_ENVIRONMENT = "us-east-1"
ASKNEWS_CLIENT_ID = "a0de4609-b760-4c83-9609-5c04d7743b84"
ASKNEWS_CLIENT_SECRET = "D5Mlhkztk4TcW24diUgcW0FA2w"
SERPER_API_KEY = "d8e815ef6caa94dbef7b977a0ea7d505b43a5a06"

# Set environment variables for AskNews
os.environ["ASKNEWS_CLIENT_ID"] = ASKNEWS_CLIENT_ID
os.environ["ASKNEWS_CLIENT_SECRET"] = ASKNEWS_CLIENT_SECRET
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

In [5]:
# Initialize the LLM and embeddings
llm = ChatGroq(temperature=0, model="llama3-8b-8192", api_key=GROQ_API_KEY)
embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

# Initialize Pinecone and vector store
pc = Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT)
pinecone_index = pc.Index("new-cyber-search")
vector_store = PineconeVectorStore(index=pinecone_index, embedding=embeddings)

# Initialize search tool
asknews_tool = AskNewsSearch(max_results=5)
google_serper = GoogleSerperAPIWrapper()
jina_search = JinaSearch()

In [7]:
class AgentState(TypedDict):
    messages: List[Dict[str, str]]
    memory: Optional[Dict[str, Any]]

def vector_search(query: str) -> str:
    results = vector_store.similarity_search(query, k=5)
    return "\n".join([f"Source {i+1}: {doc.page_content}" for i, doc in enumerate(results)])

def search_wrapper(search_func, query: str) -> str:
    results = search_func({"query": query})
    formatted_results = []
    for i, result in enumerate(results, 1):
        formatted_results.append(f"Result {i}:\nTitle: {result.get('title', 'No title')}\nSnippet: {result.get('snippet', 'No snippet')}\nURL: {result.get('link', 'No link')}")
    return "\n\n".join(formatted_results)

def execute_searches(state: AgentState) -> AgentState:
    query = state["messages"][-1]["content"]
    searches = [
        ("Vector Search", vector_search),
        ("AskNews Search", lambda q: asknews_tool.invoke({"query": q})),
        ("Google Serper Search", lambda q: search_wrapper(google_serper.run, q)),
        ("Jina Search", lambda q: search_wrapper(jina_search.invoke, q))
    ]

    for name, func in searches:
        try:
            result = func(query)
            state["messages"].append({"role": "tool", "content": f"{name} Result: {result}"})
        except Exception as e:
            state["messages"].append({"role": "tool", "content": f"{name} Error: {str(e)}"})

    return state

In [12]:
def generate_response(state: AgentState) -> AgentState:
    memory = state.get("memory", {})
    chat_history = memory.get("chat_history", "")

    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an advanced AI copilot specializing in cybersecurity and intelligence. Provide accurate, up-to-date information on cyber threats, vulnerabilities, and intelligence matters. Guidelines:

1. Directly address the query with precision and relevance.
2. Be concise and clear, using bullet points or lists when appropriate.
3. Cite sources as [Source: Name] for each piece of information.
4. Only include information from provided search results. Do not add any information from your training data.
5. If the search results don't contain relevant information to answer the query, state that you don't have enough information to provide a confident answer.
6. Specify the timeframe of information and highlight significant trends if available.
7. Maintain a neutral, professional tone and focus on defense and mitigation strategies.

Previous conversation:
{chat_history}

New human question: {input}
Vector Search Result: {vector_result}
AskNews Search Result: {asknews_result}
Google Serper Search Result: {google_serper_result}
Jina Search Result: {jina_result}

Response:"""),
    ])

    chain = prompt | llm

    response = chain.invoke({
        "input": state["messages"][-1]["content"],
        "vector_result": next((m["content"] for m in reversed(state["messages"]) if m["role"] == "tool" and "Vector Search" in m["content"]), "No vector search results."),
        "asknews_result": next((m["content"] for m in reversed(state["messages"]) if m["role"] == "tool" and "AskNews" in m["content"]), "No AskNews results."),
        "google_serper_result": next((m["content"] for m in reversed(state["messages"]) if m["role"] == "tool" and "Google Serper" in m["content"]), "No Google Serper search results."),
        "jina_result": next((m["content"] for m in reversed(state["messages"]) if m["role"] == "tool" and "Jina Search" in m["content"]), "No Jina search results."),
        "chat_history": chat_history
    })

    state["messages"].append({"role": "assistant", "content": response.content})
    state["memory"] = {"chat_history": chat_history + f"\nHuman: {state['messages'][-2]['content']}\nAI: {response.content}"}
    return state

workflow = StateGraph(AgentState)
workflow.add_node("execute_searches", execute_searches)
workflow.add_node("generate_response", generate_response)
workflow.add_edge("execute_searches", "generate_response")
workflow.add_edge("generate_response", END)
workflow.set_entry_point("execute_searches")
graph = workflow.compile()

def run_agent(query: str, memory: Optional[Dict[str, Any]] = None) -> AgentState:
    state = AgentState(messages=[{"role": "human", "content": query}], memory=memory or {})
    return graph.invoke(state)

In [13]:
if __name__ == "__main__":
    query = "Latest Cyber Incidents happened by Blackbasta Ransomeware Gang?"
    result = run_agent(query)
    for message in result["messages"]:
        if message["role"] == "assistant":
            print("AI Copilot:", message["content"])

AI Copilot: I've analyzed the provided search results and identified the following key points related to cyber threats and vulnerabilities:

1. **Ransomware attacks**: The search results highlight the increasing threat of ransomware attacks, with 67% of healthcare organizations affected in the past year. LockBit, Black Basta, and Play are among the most prolific ransomware gangs.
2. **Microsoft warning**: Microsoft has issued a warning to millions of Windows users, advising them to switch to Microsoft Edge to stop new attacks. The attacks use defense evasion tactics and rely on fraudulent websites to harvest user credentials.
3. **Ransomware in healthcare**: Sophos Limited reported a 67% increase in ransomware attacks on global healthcare organizations in 2024, with 95% of affected organizations saying the cybercriminals attempted to compromise their backups.
4. **Ransomware gangs**: ESET research shed light on CosmicBeetle, a small-time ransomware gang targeting small and midsize busi